In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pypaz

# 表情の推定
Reference from:
https://memo.sugyan.com/entry/2021/04/02/005434

In [ ]:
import pathlib
from typing import Dict, List


import glob
from pathlib import Path

import dlib
import numpy as np
import paz.processors as pr
from paz.abstract import Box2D
from paz.backend.image import load_image
from paz.pipelines import MiniXceptionFER


class EmotionDetector(pr.Processor):  # type: ignore
    def __init__(self) -> None:
        super(EmotionDetector, self).__init__()
        self.detector = dlib.get_frontal_face_detector()
        self.crop = pr.CropBoxes2D()
        self.classify = MiniXceptionFER()

    def call(self, image: np.ndarray) -> List[np.ndarray]:
        detections, scores, _ = self.detector.run(image, 1)
        boxes2D = []
        for detection, score in zip(detections, scores):
            boxes2D.append(
                Box2D(
                    [
                        detection.left(),
                        detection.top(),
                        detection.right(),
                        detection.bottom(),
                    ],
                    score,
                )
            )
        results = []
        for cropped_image in self.crop(image, boxes2D):
            results.append(self.classify(cropped_image)["scores"])
        return results


def predict(target_dir: pathlib.Path) -> Dict[str, np.ndarray]:
    results = {}
    detect = EmotionDetector()
    for i, img_file in enumerate(map(str, target_dir.glob("*.png"))):
    #for i, img_file in enumerate(glob.glob(r'/content/drive/MyDrive/seedimg/*.png')):
        image = load_image(img_file)
        predictions = detect(image)
        if len(predictions) != 1:
            continue

        print(f"{i:05d} {img_file}", predictions[0][0].tolist())
        results[img_file] = predictions[0][0]

    return results

In [ ]:
outcome['/content/drive/MyDrive/hamano.png']

array([0.0116089 , 0.04837022, 0.03210527, 0.8600259 , 0.00575359,
       0.02511219, 0.01702402], dtype=float32)

In [ ]:
#seedimgはseedで生成した画像のフィルタです
outcome = predict(Path('/content/drive/MyDrive/seedimg/'))

In [ ]:
temp = '/content/drive/MyDrive/seedimg/seed1100.png'
print(temp[-12:])

seed1100.png


In [ ]:
smile = []
not_smile = []
for key,value in outcome.items():
  if max(outcome[key]) == outcome[key][3]:
    smile.append(key[-12:])
  else:
    not_smile.append(key[-12:])

In [ ]:
len(smile)

1329

In [ ]:
'seed_'+str(int(smile[1][-8:-4])-600)+'.npz'

'seed_1504.npz'

In [ ]:
smile_npz = []
not_smile_npz = []
for i in smile:
  smile_npz.append('seed_'+str(int(i[-8:-4])-600)+'.npz')


In [ ]:
#　笑顔画像の番号を記録する
import os
txtName = "seeds.txt"
f=open(txtName,'a')
for i in smile_npz:
  f.write(i)
  f.write('\n')
f.close()

In [ ]:
import numpy as np
data = np.load('/content/drive/MyDrive/seedimg/seed_1000.npz')

In [ ]:
data['w'].shape

(1, 1, 18, 512)

In [ ]:
data['w'][0][0]
smile_avg = data['w'][0][0][0]
smile_avg.shape
temp = smile_avg*0 + smile_avg


In [ ]:
smile_avg = data['w'][0][0][0] * 0
for i in smile_npz:
  temp = np.load('/content/drive/MyDrive/seedimg/'+i)
  smile_avg =smile_avg + temp['w'][0][0][0]
smile_avg = smile_avg/1329

In [ ]:
not_smile
not_smile_npz = []
for i in not_smile:
  not_smile_npz.append('seed_'+str(int(i[-8:-4])-600)+'.npz')

662

In [ ]:
not_smile_all = data['w'][0][0][0] * 0
for i in not_smile_npz:
  temp = np.load('/content/drive/MyDrive/seedimg/'+i)
  not_smile_all =not_smile_all + temp['w'][0][0][0]
not_smile_all

array([ 6.21505318e+01,  5.00482178e+01,  9.82091675e+01, -3.37944031e+01,
       -7.84446716e+01,  2.76988010e+01, -6.12979240e+01, -4.15833168e+01,
        2.21499603e+02, -7.00213928e+01,  1.33826324e+02, -2.03234253e+01,
        1.18741875e+01,  9.28973541e+01, -8.60543060e+01, -2.07413921e+01,
       -1.14215097e+01, -1.62690220e+01,  8.02087097e+01, -9.97976589e+00,
       -2.97687130e+01, -1.24807129e+01, -6.27731819e+01,  1.41685629e+00,
        1.36204697e+02,  2.69226440e+02, -2.83357563e+01, -2.05327911e+01,
        7.81706924e+01,  6.33586407e+00,  4.60414124e+01,  2.03273201e+01,
        1.72174469e+02,  7.25789738e+00,  5.81889381e+01,  1.33770691e+02,
        1.00225876e+02,  7.85363693e+01, -5.78880157e+01,  1.32936447e+02,
        9.70195847e+01,  1.91883972e+02, -3.45625839e+01,  1.37792862e+02,
        2.53766983e+02, -2.26040668e+01,  1.10716400e+02,  2.03825188e+01,
        2.03817581e+02, -6.71275253e+01,  9.40609512e+01,  8.22564697e+01,
        2.43323608e+02, -

In [ ]:
all2smile = smile_avg - (smile_avg*1328+not_smile_all)/2000
nor2smile = smile_avg - (not_smile_all/662)

In [ ]:
print(str(all2smile[0]))

0.0015608594


In [ ]:
all2smile[0]

0.0015608594

In [ ]:
txtName = "all2smile.txt"
f=open(txtName,'a')
for i in all2smile:
  f.write(str(i))
  f.write('\n')
f.close()

txtName = "nor2smile.txt"
f=open(txtName,'a')
for i in nor2smile:
  f.write(str(i))
  f.write('\n')
f.close()

In [ ]:
coeffs = [-5.,-4.,-3.,-2.,-1.,0.,1.,2.,3.,4.]

In [ ]:
hamano_npz = np.load('/content/drive/MyDrive/projected_w.npz')


hamano_npz['w'].shape


(1, 18, 512)

In [ ]:
print(type(hamano_npz))
print(type(w))

<class 'numpy.lib.npyio.NpzFile'>
<class 'numpy.ndarray'>


In [ ]:
s = hamano_npz['w']*0
s1 = hamano_npz['w'] + -5 * all2smile
s2 = hamano_npz['w'] + -4 * all2smile
s3 = hamano_npz['w'] + -3 * all2smile
s4 = hamano_npz['w'] + -2 * all2smile
s5 = hamano_npz['w'] + -1 * all2smile
s6 = hamano_npz['w'] + 1 * all2smile
s7 = hamano_npz['w'] + 2 * all2smile
s8 = hamano_npz['w'] + 3 * all2smile
s9 = hamano_npz['w'] + 4 * all2smile
s10 = hamano_npz['w'] + 5 * all2smile


In [ ]:
np.savez('/content/drive/MyDrive/s1.npz',w=s1)
np.savez('/content/drive/MyDrive/s2.npz',w=s2)
np.savez('/content/drive/MyDrive/s3.npz',w=s3)
np.savez('/content/drive/MyDrive/s4.npz',w=s4)
np.savez('/content/drive/MyDrive/s5.npz',w=s5)
np.savez('/content/drive/MyDrive/s6.npz',w=s6)
np.savez('/content/drive/MyDrive/s7.npz',w=s7)
np.savez('/content/drive/MyDrive/s8.npz',w=s8)
np.savez('/content/drive/MyDrive/s9.npz',w=s9)
np.savez('/content/drive/MyDrive/s10.npz',w=s10)


In [ ]:
w = s8
np.savez('/content/drive/MyDrive/s1.npz',w=w)

In [ ]:
!pip install click requests tqdm pyspng ninja imageio-ffmpeg==0.4.3

     |████████████████████████████████| 204kB 26.7MB/s 
     |████████████████████████████████| 112kB 45.9MB/s 
     |████████████████████████████████| 26.9MB 118kB/s 


In [ ]:
!python '/content/drive/MyDrive/stylegan2-ada-pytorch/generate.py' --outdir='/content/drive/MyDrive/result_org' \
--projected-w='/content/drive/MyDrive/s2.npz' --network='/content/drive/MyDrive/ffhq.pkl'

Loading networks from "/content/drive/MyDrive/ffhq.pkl"...
Generating images from projected W "/content/drive/MyDrive/s1.npz"
18
512
-----------
Setting up PyTorch plugin "bias_act_plugin"... Done.
Setting up PyTorch plugin "upfirdn2d_plugin"... Done.


In [ ]:
!python '/content/drive/MyDrive/stylegan2-ada-pytorch/generate.py' --outdir='/content/drive/MyDrive/result_org' \
--projected-w='/content/drive/MyDrive/s2.npz' --network='/content/drive/MyDrive/ffhq.pkl'
!python '/content/drive/MyDrive/stylegan2-ada-pytorch/generate.py' --outdir='/content/drive/MyDrive/result_org' \
--projected-w='/content/drive/MyDrive/s3.npz' --network='/content/drive/MyDrive/ffhq.pkl'
!python '/content/drive/MyDrive/stylegan2-ada-pytorch/generate.py' --outdir='/content/drive/MyDrive/result_org' \
--projected-w='/content/drive/MyDrive/s4.npz' --network='/content/drive/MyDrive/ffhq.pkl'
!python '/content/drive/MyDrive/stylegan2-ada-pytorch/generate.py' --outdir='/content/drive/MyDrive/result_org' \
--projected-w='/content/drive/MyDrive/s5.npz' --network='/content/drive/MyDrive/ffhq.pkl'
!python '/content/drive/MyDrive/stylegan2-ada-pytorch/generate.py' --outdir='/content/drive/MyDrive/result_org' \
--projected-w='/content/drive/MyDrive/s6.npz' --network='/content/drive/MyDrive/ffhq.pkl'
!python '/content/drive/MyDrive/stylegan2-ada-pytorch/generate.py' --outdir='/content/drive/MyDrive/result_org' \
--projected-w='/content/drive/MyDrive/s7.npz' --network='/content/drive/MyDrive/ffhq.pkl'
!python '/content/drive/MyDrive/stylegan2-ada-pytorch/generate.py' --outdir='/content/drive/MyDrive/result_org' \
--projected-w='/content/drive/MyDrive/s8.npz' --network='/content/drive/MyDrive/ffhq.pkl'

!python '/content/drive/MyDrive/stylegan2-ada-pytorch/generate.py' --outdir='/content/drive/MyDrive/result_org' \
--projected-w='/content/drive/MyDrive/s9.npz' --network='/content/drive/MyDrive/ffhq.pkl'
!python '/content/drive/MyDrive/stylegan2-ada-pytorch/generate.py' --outdir='/content/drive/MyDrive/result_org' \
--projected-w='/content/drive/MyDrive/s10.npz' --network='/content/drive/MyDrive/ffhq.pkl'

Loading networks from "/content/drive/MyDrive/ffhq.pkl"...
Generating images from projected W "/content/drive/MyDrive/s2.npz"
18
512
-----------
Setting up PyTorch plugin "bias_act_plugin"... Done.
Setting up PyTorch plugin "upfirdn2d_plugin"... Done.
Loading networks from "/content/drive/MyDrive/ffhq.pkl"...
Generating images from projected W "/content/drive/MyDrive/s3.npz"
18
512
-----------
Setting up PyTorch plugin "bias_act_plugin"... Done.
Setting up PyTorch plugin "upfirdn2d_plugin"... Done.
Loading networks from "/content/drive/MyDrive/ffhq.pkl"...
Generating images from projected W "/content/drive/MyDrive/s4.npz"
18
512
-----------
Setting up PyTorch plugin "bias_act_plugin"... Done.
Setting up PyTorch plugin "upfirdn2d_plugin"... Done.
Loading networks from "/content/drive/MyDrive/ffhq.pkl"...
Generating images from projected W "/content/drive/MyDrive/s5.npz"
18
512
-----------
Setting up PyTorch plugin "bias_act_plugin"... Done.
Setting up PyTorch plugin "upfirdn2d_plugin"..

In [ ]:
import glob
from PIL import Image

# filepaths
fp_in = "/content/drive/MyDrive/result_org/*.png"
fp_out = "/content/drive/MyDrive/result_org/image.gif"

# https://pillow.readthedocs.io/en/stable/handbook/image-file-formats.html#gif
img, *imgs = [Image.open(f) for f in sorted(glob.glob(fp_in))]
img.save(fp=fp_out, format='GIF', append_images=imgs,
         save_all=True, duration=200, loop=0)

In [ ]:
smile_npz

['seed_1501.npz',
 'seed_1504.npz',
 'seed_1506.npz',
 'seed_1507.npz',
 'seed_1508.npz',
 'seed_1509.npz',
 'seed_1510.npz',
 'seed_1511.npz',
 'seed_1512.npz',
 'seed_1513.npz',
 'seed_1514.npz',
 'seed_1515.npz',
 'seed_1516.npz',
 'seed_1517.npz',
 'seed_1519.npz',
 'seed_1520.npz',
 'seed_1521.npz',
 'seed_1522.npz',
 'seed_1523.npz',
 'seed_1525.npz',
 'seed_1526.npz',
 'seed_1527.npz',
 'seed_1529.npz',
 'seed_1530.npz',
 'seed_1532.npz',
 'seed_1533.npz',
 'seed_1536.npz',
 'seed_1537.npz',
 'seed_1539.npz',
 'seed_1545.npz',
 'seed_1546.npz',
 'seed_1549.npz',
 'seed_1551.npz',
 'seed_1552.npz',
 'seed_1553.npz',
 'seed_1554.npz',
 'seed_1555.npz',
 'seed_1558.npz',
 'seed_1562.npz',
 'seed_1563.npz',
 'seed_1564.npz',
 'seed_1565.npz',
 'seed_1566.npz',
 'seed_1567.npz',
 'seed_1568.npz',
 'seed_1569.npz',
 'seed_1571.npz',
 'seed_1572.npz',
 'seed_1574.npz',
 'seed_1575.npz',
 'seed_1578.npz',
 'seed_1579.npz',
 'seed_1581.npz',
 'seed_1582.npz',
 'seed_1583.npz',
 'seed_158

In [ ]:

data = []
for i, file in enumerate(map(str, Path('/content/drive/MyDrive/seedimg/').glob("*.npz"))):
  #print(file.split('/')[5])
  temp_data = (np.load(file))['w'][0][0][0]
  #print(temp_data)
  if (file.split('/')[5]) in smile_npz:
    temp_data = np.append(temp_data,1)
  else:
    temp_data = np.append(temp_data,0)
  data.append(temp_data)


In [ ]:
np.savetxt("/content/drive/MyDrive/temp.csv", data, delimiter=",")

In [ ]:
import pandas as pd

import numpy as np

from sklearn.model_selection import train_test_split

from imblearn.under_sampling import RandomUnderSampler

from sklearn.preprocessing import StandardScaler

from sklearn.svm import SVC

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import accuracy_score

from sklearn.metrics import confusion_matrix

from sklearn.metrics import roc_curve
from sklearn.metrics import auc

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [ ]:
df = pd.DataFrame(data)

In [ ]:
columnList = df.columns.to_list()
#columnList.remove("SystemID")
columnList.remove(512)
X = pd.DataFrame( df.loc[:, columnList] )
Y = pd.DataFrame( df.loc[:, [512]] )


In [ ]:
Y

,512
0,1.0
1,0.0
2,0.0
3,1.0
4,0.0
...,...
1996,0.0
1997,1.0
1998,1.0
1999,1.0
